In [ ]:
#Install Library First
!pip install beautifulsoup4
!pip install requests

In [ ]:
#import library
import requests
import csv
from bs4 import BeautifulSoup as bs

#copy paste NPSN sekolah dari .csv (run cloud)
list_npsn = [20219178,20219146,20219146,20219145,20219144,20219143,20219142,20219141,20219140,20219139,20219176,20219175,20219174,20219161,20219160,20219159,20219137,20219148,20219158,20265543,20219295]

#Make Array for exporting csv
data_export = []

### Fungsi untuk cari sekolah berdasarkan NPSN
# input NPSN tipe integer
# output Nama sekolah dan link sekolah dalam bentuk tupple

def find_url_and_school_name(npsn):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.76'
    }
    payload = {
        'page':1,
        'kode_kabupaten':'',
        'kode_kecamatan':'',
        'bentuk_pendidikan':'',
        'status_sekolah':'semua',
        'nama':npsn
        }
    try:
      session = requests.Session()
      response = session.post('http://sekolah.data.kemdikbud.go.id/index.php/Chome/pencarian/',headers=headers,data=payload)
      soup = bs(response.content,'html.parser')
      a = soup.find('ul',attrs={'class':'list-group list-group-unbordered'}).find('li',attrs={'class':'list-group-item'}).find('a')
      nama_sekolah = a.text
      link_sekolah = a.attrs['href']
    except:
      print("ada yang error bro")
    return nama_sekolah,link_sekolah

#array untuk data sekolah yang akan di scrap
data_sekolah = []
error_log = []

#Get All school list and Link
for no_npsn in list_npsn:
  for l in range(0,2):
    data_sekolah.append(find_url_and_school_name(no_npsn)[l])

#Initial Value
print(data_sekolah[0])
f = 0
e = 0
error_message = 'Data Tidak Ditemukan'

#Scrapping per School
for a in range(1,len(data_sekolah)+1,2):
  #Get url
  res = requests.get(data_sekolah[a])

  #Make object
  soup = bs(res.content)

  #Array fix
  data_fix = []

  #Input Nama Sekolah
  data_fix.append(data_sekolah[e])
  e = e+2

  #Get Kepala Sekolah and Operator
  data = soup.find_all('li', class_="list-group-item")
  data2 = []
 
  for i in data:
    data2.append(str(i.text))
    
  print(data2)
  #if "Kepala Sekolah" and "Operator" in data2:
  #Processing String
  col1 = data2[2]
  col2 = data2[3]
  col1.lstrip()
  col2.lstrip()
  kepsek = col1[19:]
  operator = col2[13:]  

  #Input kepsek dan operator ke data_fix
  data_fix.append(kepsek)
  data_fix.append(operator)

  # else :
  #   error_log.append("NULL")
  #   data_fix.append(error_message)
  #   data_fix.append(error_message)

  #Search Siswa dan Rombel
  array = []
  for id in soup.find_all(id="siswatingkat"):
    for td in id.find_all('td'):
      array.append(str(td.text))

  #Input Data Jumlah Siswa (urut kelas 10 hingga 12)
  for index in range(5,10,2):
    data_fix.append(array[index])

  #Input Data Rombel Kelas (urut kelas 10 hingga 12)
  for index2 in range(17,23,2):
    data_fix.append(array[index2])

  #Jumlah Siswa Laki-laki dan Perempuan
  data = soup.find_all('font',class_='text-info')
  data2 = []
  for i in data:
    data2.append(str(i.text))

  #Input Siswa Laki-laki dan Perempuan
  siswa_male = data2[1]
  data_fix.append(siswa_male)
  siswa_female = data2[2]
  data_fix.append(siswa_female)

  #Input Data Semester
  semester_data = data2[5]
  data_fix.append(semester_data)

  #Data Akreditasi
  data = soup.find_all('li',class_='list-group-item-info')
  data2 = []
  for i in data:
    data2.append(str(i.text))
  akreditasi = data2[0]  

  #Input data Akreditasi
  akreditasi = akreditasi[-1]
  data_fix.append(akreditasi)

  #Data Alamat
  data = soup.find_all('font',class_='small')
  data2 = []
  for i in data:
    data2.append(str(i.text))
  alamat = data2[0]

  #Input data Alamat
  alamat = alamat[:-19]
  data_fix.append(alamat)

  #Data Email Website
  try:
    b = f"https://referensi.data.kemdikbud.go.id/tabs.php?npsn={list_npsn[f]}"
    f = f+1
    res2 = requests.get(b)
    soup2 = bs(res2.content,'html.parser')
    array = [] 
    email = [] 
    website = []
    sub_mail = '@'
    sub_web = 'http'
    for id in soup2.find_all(id="tabs-6"):
      for td in id.find_all('td'):
        array.append(str(td.text))
        email = [c for c in array if sub_mail in c]
        website = [d for d in array if sub_web in d]  

    #Input Email dan Website
    data_fix.append(email[0])
    data_fix.append(website[0])

  except:
    error_log.append(b)
    data_fix.append('Email Tidak Ditemukan')
    data_fix.append('Website Tidak Ditemukan')



  #write data to prepare exporting
  data_export.append(data_fix)

#Cek data akhir (opsional)
print(data_fix)
print(data_export)


#Write down data to csv
header = ['Nama Sekolah','Kepala Sekolah','Operator','Jumlah Siswa Kelas 10','Jumlah Siswa Kelas 11', 'Jumlah Siswa Kelas 12', 'Total Kelas 10', 'Total Kelas 11', 'Total Kelas 12', 'Jumlah Siswa Laki-laki', 'Jumlah Siswa Perempuan', 'Data Semester', 'Akreditasi','Alamat','Email','Website']
with open("data_scrapping.csv","w",newline="") as files:
  writer = csv.writer(files)
  writer.writerow(header)
  writer.writerows(data_export)